# Crimson Responses - kilosort

basically the same thing as the other crimson response stuff, except we're working with kilosort stuff

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from scipy.interpolate import RBFInterpolator
from sklearn.decomposition import PCA
from sklearn.linear_model import PoissonRegressor
from scipy.optimize import curve_fit
import os, glob, re
import openephys_utils 
from tqdm.notebook import tqdm
import kilosort
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Polygon
import csv
from scipy.stats import ttest_ind

%matplotlib qt


Get firing rates from kilosort data -- since this will need to work differently from the version I did with the "spike dictionary"

In [2]:
def kilosort_FR(kilosort_dir:str, bin_sec:float = .01, fs:int = 30000):
    '''
    Return the firing rates for each template

    inputs:
        kilosort_dir:str        - directory where the kilosort results were stored
        bin_sec:float           - bin length in seconds [.01]
        fs:int                  - sampling rate [30000]

    outputs:
        firing_rates:np.array   - TxN array of firing rates of N templates
    '''
    spike_times = np.load(os.path.join(kilosort_dir, 'spike_times.npy'))
    spike_templates = np.load(os.path.join(kilosort_dir, 'spike_templates.npy'))

    # bin times defined by sample #s, not seconds
    bin_times = np.arange(np.ceil(spike_times[-1]), step=bin_sec*fs)
    # empty array to fill later
    firing_rates = np.ndarray((bin_times.shape[0]-1, np.unique(spike_templates).shape[0]))
    for template in np.unique(spike_templates): # for each template...
        firing_rates[:,template] = np.histogram(spike_times[spike_templates == template], bins=bin_times)[0]

    return firing_rates/bin_sec, bin_times[:-1]/fs




    

batch run through kilosort on filtered data

october 2nd

In [24]:
# get a list of directories
base_dir = 'Z:\\BrainPatch\\20241002\\lateral\\'
directories = [os.path.join(base_dir, directory) for directory in os.listdir(base_dir) if ('2ms' in directory and '15mA' in directory)]

# probe and settings
probe_name = 'Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat'
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':0}


September 25

In [60]:
# get a list of directories
base_dir = 'Z:\\BrainPatch\\20240925\\'
directories = [os.path.join(base_dir, directory) for directory in os.listdir(base_dir) if ('2ms' in directory and '15mA' in directory)]

# probe and settings
probe_name = "Z:\\BrainPatch\\20240925\\64-4shank-poly-brainpatch-chanMap.mat"
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}


rename mm to um in directory name if needed

In [ ]:
# directories = [os.path.join('Z:\\\\BrainPatch\\\\20240718', directory) for directory in os.listdir('Z:\\BrainPatch\\20240718') if 'mm' in directory]
directories = [os.path.join(base_dir, directory) for directory in os.listdir(base_dir) if 'mm' in directory]

for directory in directories:
    distance = re.search('(\d?\.?\d)mm',directory)

    dist_mult = int(float(distance[1]) * 1000)
    # print(f'{directory[:distance.span()[0]]}{dist_mult:03d}um{directory[distance.span()[1]:]}')
    rename_dir = f'{directory[:distance.span()[0]]}{dist_mult:03d}um{directory[distance.span()[1]:]}'

    os.rename(directory, rename_dir)



In [3]:
def gimme_a_FR_df(directories, probe_name, settings):
    # open the probe
    probe = kilosort.io.load_probe(probe_name)


    # sos for a BPF
    filtered = True # should we filter the data?
    sos = signal.butter(N = 8, Wn = [300, 6000], btype='bandpass', fs=30000, output='sos') if filtered else None

    # binning decisions
    bin_sec = 0.002

    # to create a pandas array later I guess
    FR_list = [] # this will contain the pre-stim mean, the mean stim response, and the map info

    # length of interest for the responses and the "low mean firing rate"
    resp_len = 10 # in bins
    low_fr_cutoff = 1


    # for use later
    color_inds_map = ['low_firing','inhibited','excited']

    for directory in tqdm(directories):

        # pull out the current and distance
        current = int(re.search('(\d{1,2})mA', directory)[1])
        distance = int(re.search('(\d{3,4})um', directory)[1])

        distances = np.sqrt(distance**2 + (probe['xc'] - probe['xc'].mean())**2)

        # print it all out
        print(f'{current}mA {distance}mm')

        # get sig_eraasr if it doesn't exist already
        # eraasr_path = os.path.join(directory, 'sig_eraasr.npy')
        # if not os.path.exists(eraasr_path):
        #     sig, timestamps, stims, stim_ts = openephys_utils.open_sig_stims(directory)

        #     # clean, pull out threshold crossings, get firing rates
        #     sig_eraasr = openephys_utils.ERAASR(sig, save_dir=directory)
        # else:
        #     sig_eraasr = np.load(eraasr_path)

        # filter at 300, 6000
        if filtered:
            filt_path = os.path.join(directory, 'sig_filter.npy')
            if not os.path.exists(filt_path):
                sig_filter = signal.sosfiltfilt(sos, sig_eraasr, axis=0)
                np.save(filt_path, sig_filter)


        # kilosort
        if not filtered:
            # looking at the eraasr location
            eraasr_path = os.path.join(directory, 'sig_eraasr.npy')
            results_dir=os.path.join(directory, 'kilosort4_unfiltered')
            
            if not os.path.exists(results_dir): # run it if it doesn't already exist. Otherwise just use the existing
                if not os.path.exists(eraasr_path):
                    sig, timestamps, stims, stim_ts = openephys_utils.open_sig_stims(directory)
                    # clean, pull out threshold crossings, get firing rates
                    sig_eraasr = openephys_utils.ERAASR(sig, save_dir=directory)
                else:
                    sig_eraasr = np.load(eraasr_path)
                
                settings['filename'] = eraasr_path
                kilosort.run_kilosort(settings, file_object=sig_eraasr.astype(np.float32), data_dtype='float32', results_dir=results_dir)

            else:
                print(f'{results_dir} already exists, using existing files')

        else:
            filt_path = os.path.join(directory, 'sig_filter.npy')
            results_dir=os.path.join(directory, 'kilosort4_filtered')

            if not os.path.exists(results_dir): # run it if it doesn't already exist. Otherwise just use the existing
                if not os.path.exists(filt_path):
                    sig, timestamps, stims, stim_ts = openephys_utils.open_sig_stims(directory)
                    # clean, pull out threshold crossings, get firing rates
                    sig_eraasr = openephys_utils.ERAASR(sig, save_dir=directory)
                    sig_filter = signal.sosfiltfilt(sos, sig_eraasr, axis=0)
                else:
                    sig_filter = np.load(filt_path)

                settings['filename'] = os.path.join(directory,'sig_filter.npy')
                kilosort.run_kilosort(settings, file_object=sig_filter.astype(np.float32), data_dtype='float32', results_dir=results_dir)

            else:
                print(f'{results_dir} already exists, using existing files')

        # firing rates from kilosort
        firing_rate, bins = kilosort_FR(results_dir, bin_sec=bin_sec)

        # get the channel-to-template map
        template_wf = np.load(os.path.join(results_dir, 'templates.npy'))
        channel_best = (template_wf**2).sum(axis=1).argmax(axis=-1) # find the channel with biggest variance

        # get stimulation times in bin count
        stims_bin = (stims/(30000*bin_sec)).astype(int)

        # pre-stimulation data
        prestim_means = firing_rate[:stims_bin[0,0], :].mean(axis=0) # put into firing rates instead of counts
        prestim_std = firing_rate[:stims_bin[0,0], :].std(axis=0) # put into firing rates instead of counts

        # get the means for 5 bins after the stims
        # indices for 5 ms after end of stimulation
        poststim_inds = np.array([np.arange(start=stim, stop=stim+resp_len) for stim in stims_bin[:,1]]).flatten()
        # split out that portion of the array and reshape it to a Stims x Time x Templates array
        poststim_resp = firing_rate[poststim_inds,:].reshape((stims.shape[0], resp_len, firing_rate.shape[1]))
        # find the mean
        poststim_mean = poststim_resp.mean(axis=0)

        # # split into three colors -- minimal fr before, fr increases, fr decreases
        color_inds = 1 + (poststim_mean.mean(axis=0)>prestim_means).astype(int)
        color_inds[prestim_means < low_fr_cutoff] = 0

        # # colors for depending on change of firing rate

        # fig,ax = plt.subplots(nrows = 3, sharex=True)

        # # colors = ['blue','red','black']
        # for channel in np.arange(prestim_means.shape[0]):
        #     ax[color_inds[channel]].plot(np.arange(poststim_mean.shape[0])*bin_sec*1000, (poststim_mean[:,channel] - prestim_means[channel]))


        # ax[0].set_title('Low initial firing')
        # ax[0].set_ylabel('Firing Rate (hz)')
        # ax[1].set_title('Decreased firing')
        # ax[1].set_ylabel('Firing Rate (hz)')
        # ax[2].set_title('Increased firing')
        # ax[2].set_ylabel('Firing Rate (hz)')
        # ax[2].set_xlabel('Time (ms)')

        # fig.savefig(os.path.join(directory, 'FR_changes.png'))

        # plt.close(fig)

        # is it significantly higher FR than previously


        for template,channel in enumerate(channel_best):
            significant = ttest_ind(firing_rate[:stims_bin[0,0], template], poststim_resp[:,0,template], equal_var = False, alternative = 'less')
            
            
            FR_info = {'template':template,
                    'channel': channel,
                    'prestim_mean': prestim_means[template],
                    'prestim_std' : prestim_std[template],
                    'poststim_max': poststim_mean[:,template].max(),
                    'poststim_first':poststim_mean[0,template],
                    'type': color_inds_map[color_inds[template]],
                    'distance': distances[channel],
                    'depth': -probe['yc'][channel],
                    'current': current,
                    'shank_no':probe['kcoords'][channel],
                    'template_wf':template_wf[template,:,channel],
                    'mean_firing_rate':poststim_mean[:,template],
                    'recording':directory,
                    'significant':significant}
        
            FR_list.append(FR_info)


    FR_df = pd.DataFrame(FR_list)

    return FR_df

normalize the firing rates in different ways

In [5]:
FR_df['zscore'] = (FR_df['poststim_max']-FR_df['prestim_mean'])/FR_df['prestim_std'] 
FR_df['fold_change'] = (FR_df['poststim_max']/FR_df['prestim_mean'])

Let's take a look at a histogram of the firing rates during the first 2 ms

In [67]:
fig_immediate, ax_immediate = plt.subplots()

ax_immediate.hist(FR_df['poststim_first'], bins=50)

ax_immediate.set_xlabel('Mean firing rate (hz)')
ax_immediate.set_ylabel('Number of units')
ax_immediate.set_title('Post-stimulation mean firing rate (hz, 2ms window)')

for spine in ax_immediate.spines:
    ax_immediate.spines[spine].set_visible(False)

Plot the distance vs the peak firing rate and the "fold change" peak firing rate

In [ ]:
# get the means of the "excited" 
excited_df = FR_df.loc[FR_df['type'] == 'excited'].groupby(['distance', 'depth']).mean('poststim_max', 'zscore','fold_change')

# pull out the depth, distance, and means
indices = np.array(excited_df.index.values.tolist()) # [depth, distance]
max_mu = excited_df['poststim_max'].values # [mean]

# step size for the mesh grid
mstep = 30

# create a meshgrid for the depth and distances
dd_mg = np.mgrid[int(indices[:,0].min()):int(indices[:,0].max()+mstep):mstep, int(indices[:,1].min()):int(indices[:,1].max()+mstep):mstep]
dd_mg_reshape = dd_mg.reshape(2,-1).T


# working with the peak firing rates

# interpolate
interpolation = RBFInterpolator(indices,max_mu)(dd_mg_reshape)

# put up the color map
fig_peak,ax_peak = plt.subplots()
pos = ax_peak.pcolormesh(*dd_mg, interpolation.reshape(dd_mg.shape[1:]))

# show the electrode locations 
ax_peak.scatter(indices[:,0], indices[:,1], color='white')

# make the plot pretty
ax_peak.set_xlabel('Horizontal distance between recording electrode and LED ($\mu$m)')
ax_peak.set_ylabel('Depth of recording electrode ($\mu$m)')
ax_peak.set_title('Maximum firing rates')
for spine in ax_peak.spines:
    ax_peak.spines[spine].set_visible(False)

# put a colorbar on there
fig_peak.colorbar(pos, ax=ax_peak)


# peak FR divided by the pre-stim mean

fold_mus = excited_df['fold_change'].values

# interpolate
interpolation_fold = RBFInterpolator(indices,fold_mus)(dd_mg_reshape)

# put up the color map
fig_fold,ax_fold = plt.subplots()
pos_fold = ax_fold.pcolormesh(*dd_mg, interpolation_fold.reshape(dd_mg.shape[1:]))

# show the electrode locations 
ax_fold.scatter(indices[:,0], indices[:,1], color='white')

# make the plot pretty
ax_fold.set_xlabel('Horizontal distance between recording electrode and LED ($\mu$m)')
ax_fold.set_ylabel('Depth of recording electrode ($\mu$m)')
ax_fold.set_title('Maximum fold change')
for spine in ax_fold.spines:
    ax_fold.spines[spine].set_visible(False)

# put a colorbar on there
fig_peak.colorbar(pos_fold, ax=ax_fold)


# peak FR divided by the pre-stim mean

z_mus = excited_df['zscore'].values

# interpolate
interpolation_z = RBFInterpolator(indices,z_mus)(dd_mg_reshape)

# put up the color map
fig_z,ax_z = plt.subplots()
pos_z = ax_z.pcolormesh(*dd_mg, interpolation_z.reshape(dd_mg.shape[1:]))

# show the electrode locations 
ax_z.scatter(indices[:,0], indices[:,1], color='white')

# make the plot pretty
ax_z.set_xlabel('Horizontal distance between recording electrode and LED ($\mu$m)')
ax_z.set_ylabel('Depth of recording electrode ($\mu$m)')
ax_z.set_title('Maximum z change')
for spine in ax_z.spines:
    ax_z.spines[spine].set_visible(False)

# put a colorbar on there
fig_peak.colorbar(pos_z, ax=ax_z)

Try a multi-subplot version, with depth and distance on separate plots. For this we'll use the "init" values, and use all values not just the "excited" ones

First we have to define a the curve we're fitting and a jacobian (assume that'll make it easier). I'm thinking we'll do a sin function with some sort of exponential for the depth, and a decaying exponential for the distance

In [76]:
# np.sin(depth) + np.exp(distance)
# def dd_curve(inputs, p0, p1, p2, p3, p4, p5):
# def dd_curve(inputs, p0, p1, p2, p3):
def dd_curve(inputs, p0, p1, p2, ):
    # return p0*np.sin(p1*inputs[:,0] + p2) + p3*np.exp(p4 * inputs[:,1]) + p5
    # return p0*np.exp(inputs[:,0] * p1) + p2*inputs[:,1]  + p3
    return p0*inputs[:,0] + p1*inputs[:,1] + p2



In [78]:
grid = plt.GridSpec(nrows=3, ncols=3, wspace=.3, hspace=.3)

fig_combo = plt.figure()
fig_combo.set_size_inches((8,8))

ax_heatmap = fig_combo.add_subplot(grid[:2, 1:])
ax_depth = fig_combo.add_subplot(grid[:2,0], sharey = ax_heatmap)
# ax_heatmap.set_aspect('equal')
ax_dist = fig_combo.add_subplot(grid[2,1:], sharex = ax_heatmap)


# depth vs first two seconds
ax_depth.scatter(FR_df['poststim_first'],FR_df['depth'], 2)


# distance vs first two seconds
ax_dist.scatter(FR_df['distance'],FR_df['poststim_first'], 2)

# -----------------------------
# heatmap
indices = np.array(FR_df[['distance','depth']]) # [depth, distance]
# indices[:,0] = -indices[:,0]
first_vals = FR_df['poststim_first'].values 

# step size for the mesh grid
mstep = 30

# create a meshgrid for the depth and distances
mg = np.mgrid[int(FR_df['distance'].min()):int(FR_df['distance'].max()+mstep):mstep, int(FR_df['depth'].min()):int(FR_df['depth'].max()+mstep):mstep]
mg_reshape = mg.reshape(2,-1).T

# # poisson regression
# lr = PoissonRegressor().fit(indices, first_vals)
# mg_predict = lr.predict(mg_reshape)
popt, pcon = curve_fit(dd_curve, indices, first_vals)
mg_predict = dd_curve(mg_reshape, *popt)

# put up the color map
pos = ax_heatmap.pcolormesh(*mg, mg_predict.reshape(mg.shape[1:]))

# show the electrode locations 
ax_heatmap.scatter(indices[:,0], indices[:,1], color='white')

# plot the fits on the scatter plots
ax_dist.plot(mg_reshape[:,0], dd_curve(np.array([mg_reshape[:,0], np.zeros((mg_reshape.shape[0],))]).T, *popt))
ax_depth.plot(dd_curve(np.array([np.zeros((mg_reshape.shape[0],)), mg_reshape[:,1]]).T, *popt), mg_reshape[:,1])

# make the heatmap plot pretty
ax_heatmap.set_title('Post-stimulus firing rates')
for spine in ax_heatmap.spines:
    ax_heatmap.spines[spine].set_visible(False)

# distance and depth plots
ax_dist.set_xlabel('Horizontal distance between recording electrode and LED ($\mu$m)')
ax_dist.set_ylabel('Firing rate (hz, 2ms window)')
for spine in ax_dist.spines:
    ax_dist.spines[spine].set_visible(False)

ax_depth.set_ylabel('Depth of recording electrode ($\mu$m)')
ax_depth.set_xlabel('Firing rate (hz, 2ms window)')
for spine in ax_depth.spines:
    ax_depth.spines[spine].set_visible(False)

# axis insert
axins = inset_axes(ax_heatmap,
                    width="5%",  
                    height="100%",
                    loc='center right',
                    borderpad=-5
                   )

# for spine in axins.spines:
#     axins.spines[spine].set_visible(False)

# put a colorbar on there
cb = fig_combo.colorbar(pos, cax=axins)

cb.set_label('Firing rate (hz, 2ms window)')




In [ ]:
mns_df = excited_df.mean('poststim_max')

depth_im = np.ndarray((int(FR_df['depth'].max()), int(FR_df['distance'].max())))

depths = FR_df['depth'].unique()
distances = FR_df['distance'].unique()

for i_depth in np.arange(depths.shape[0]):
    for i_distance in np.arange(distances.shape[0]):
        if i_depth == 0 or i_depth == depths.shape[0] or i_distance == 0 or i_distance == distances.shape[0]:
            continue
        depth_im[int((depths[i_depth-1]+depths[i_depth])/2):int((depths[i_depth]+depths[i_depth+1])/2),int((distances[i_distance-1]+distances[i_distance])/2):int((distances[i_distance]+distances[i_distance+1])/2)] = mns_df.loc[mns_df['depth'] == depth and mns_df['distance'] == distance]['poststim_max']




different method - no interpolation or linear fits, just a square for each point

In [ ]:

# get a distance and depth grouping for the 2ms after stim
base_dirs = ['Z://BrainPatch//Raw_Data//20241002//',
             'Z://BrainPatch//Raw_Data//20240925//',
             'Z:BrainPatch//Raw_Data//20240821']
# base_dirs = ['Z:/BrainPatch/Raw_Data/20241002']

# all 2ms stimulations at 400 um in the base_dirs
directories = [os.path.join(base_dir,directory) for base_dir in base_dirs for directory in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir,directory)) and '2ms' in directory and '15mA' in directory]

# probe map
probe_name = "Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat"

# settings for kilosort
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

# get a firing rate dataframe
FR_df = gimme_a_FR_df(directories, probe_name, settings)
FR_df['depth'] = FR_df['depth'].replace(-0.0, 0)

# group it
summary = FR_df.groupby(['depth','distance'])['poststim_first'].mean()


In [ ]:
# figure
fig_patch, ax_patch = plt.subplots()

# Create a map to fill with the means
depths = FR_df['depth'].unique()
depths.sort()
depths = depths[::-1]
distances = FR_df['distance'].unique()
distances.sort()
n_depths = len(depths)
n_distances = len(distances)
layout = np.ones([2*n_depths+1,2*n_distances+1]) * np.nan

# create an x and y mapping from the distance to the x/y position on the heatmap
y_mapping = dict(zip(depths, np.arange(2*n_depths, step=2) + 1))
x_mapping = dict(zip(distances, np.arange(2*n_distances, step=2)+1))

# for each line in the summary
for index, mean in summary.items():
    # print(f'{index[0]},{index[1]}: {mean}')
    layout[y_mapping[index[0]],x_mapping[index[1]]] = mean

cma = ax_patch.imshow(layout)
ax_patch.set_xticks(np.arange(2*n_distances, step=2)+1)
ax_patch.set_xticklabels([int(distance) for distance in distances])
ax_patch.tick_params(axis='x', rotation=45)
ax_patch.set_xlabel('Distance ($\mu$m)')
ax_patch.set_yticks(np.arange(2*n_depths, step=2)+1)
ax_patch.set_yticklabels([int(depth) for depth in depths])
ax_patch.set_ylabel('Depth ($\mu$m)')

for spine in ax_patch.spines:
    ax_patch.spines[spine].set_visible(False)

fig_patch.colorbar(cma, ax=ax_patch)



In [ ]:
ax_patch.set_xticks(np.arange(2*n_distances, step=2)+1)
ax_patch.set_xticklabels(distances)
ax_patch.set_yticks(np.arange(2*n_depths, step=2)+1)
ax_patch.set_yticklabels([int(depth) for depth in depths])


In [ ]:
x_mapping

# Template waveforms for each electrode

Looks like we're wanting some waveforms, so let's get some waveforms

In [50]:
# for each channel find the template with a pre-stim mean above 5 hz with the largest waveform

# notes: in the 15mA setup, we didn't get any spikes on channels 13 and 21

wf_dict = {}
template_no = []

for channel in FR_df['channel'].unique():
    channel_subset = FR_df.loc[(FR_df['channel']==channel) & (FR_df['prestim_mean']>0.5)]

    if channel_subset.shape[0] > 0:
        biggest_template = np.stack(channel_subset['template_wf'].values).min(axis=1).argmin()
        wf_dict[channel] = channel_subset.iloc[biggest_template]['template_wf']
        template_no.append(channel_subset.iloc[biggest_template]['template'])


# store them for JingLan
waveform_csv = os.path.join(base_dir, 'best_waveforms.csv')
with open(waveform_csv, 'w') as fid:
    csv_writer = csv.writer(fid)
    for channel,waveform in wf_dict.items():
        row = f'{channel}, ,{waveform.tolist()}\n'.replace('[','').replace(']','')
        fid.write(row)
        


In [ ]:
probe_grid = plt.GridSpec(16,4, wspace=.5, hspace=.7)

fig_probe = plt.figure()

ax_probe = dict()
for i_channel, (channel,waveform) in enumerate(wf_dict.items()):
    row = int(probe['yc'][channel]/50)
    col = int(probe['kcoords'][channel]) - 1

    ax_probe[channel] = fig_probe.add_subplot(probe_grid[row,col])
    ax_probe[channel].plot(waveform)
    ax_probe[channel].set_title(f'channel')
    print(channel)

    for spine in ax_probe[channel].spines:
        ax_probe[channel].spines[spine].set_visible(False)

    ax_probe[channel].set_xticks([])
    ax_probe[channel].set_yticks([])



In [74]:
# create a PDF with a plot for each channel, with the index number for each waveform labeled
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(os.path.join(base_dir, 'all_templates.pdf')) as pdf:
    fig, ax = plt.subplots()
    for channel in FR_df['channel'].unique():
        pdf.attach_note(f'Channel {channel}')
        for index,item in FR_df.loc[FR_df['channel']==channel].iterrows():
            ax.plot(item['template_wf'], label=f"Index {index}, template {item['template']}")

        ax.set_title(channel)
        ax.legend()
        ax.set_xticks([])
        ax.set_yticks([])

        pdf.savefig(fig)
        ax.cla()

# the recommended songs after the Gizz's new single are really working for me today
# kikagagku moyo
# Haitus Koyote
# Karen O and Danger Mouse have a new album. That's cool
# The mystery lights - memories

In [85]:
# a channel-to-FR_df mapping of the waveforms we like. 
# A lot of these are consistent enough that I suspect they're the same unit across recordings.

wf_mapping = {
    41: 620, # could also be 211, 481, or 340
    34: 208, # or 745
    56: 113, # or 487
    62: 751,
    0: 350,
    6: 220,
    23: 120,
    28: 358,
    32: 232,
    45: 20,
    58: 363,
    54: 236,
    8: 370,
    19: 243,
    36:126, 
    37: 33,
    63: 255,
    1: 387,
    2: 388,
    26: None,
    38: 524,
    49: 786,
    52: 526,
    10: 149,
    3: 538,
    24: None,
    35: 286,
    39: 412,
    57: 159,
    49: 556,
    5: 818, 
    25: None,
    42: 569,
    51: 689,
    44: 570,
    50: 306,
    18: 76, # only exists in one recording, might toss
    20: None,
    43: 312,
    33: 710,
    46: 83,
    9: None,
    31: 192,
    47: 457,
    53: 195,
    40: 464,
    48: 868,
    14: 739,
    11: 335,
    17: 338,
    60: 383,
    27: None,
    61: 400,
    29: None,
    12: 576,
    4: 239,
    30: 373,
    7: 422,
    16: None,
    15: None,
    22: 736,
}

# Waveforms, laid out according to the probe mapping from NeuroNexus
probe_grid = plt.GridSpec(16,4, wspace=.5, hspace=.7)

# probe figure
fig_probe = plt.figure()
fig_probe.set_size_inches(5,10)

# keeping track of axes using a dictionary
ax_probe = dict()
# iterate through the waveform/channel mapping
for channel, index in (wf_mapping.items()):
    # if there aren't any nice waveforms on this channel
    if index is None:
        continue
    
    # row/column mapping for the probe
    row = int(probe['yc'][channel]/50)
    col = int(probe['kcoords'][channel]) - 1

    # plot in the right location
    ax_probe[channel] = fig_probe.add_subplot(probe_grid[row,col], facecolor=None)
    ax_probe[channel].plot(FR_df.iloc[index]['template_wf'])

    # get rid of the spines to make everything cleaner
    for spine in ax_probe[channel].spines:
        ax_probe[channel].spines[spine].set_visible(False)

    # get rid of the ticks -- these templates are normalized anyways
    ax_probe[channel].set_xticks([])
    ax_probe[channel].set_yticks([])


# store the waveforms in a csv for plotting in another program if wanted
waveform_csv = os.path.join(base_dir, 'example_waveforms.csv')
with open(waveform_csv, 'w') as fid:
    # header labels
    header = 'Channel, ,Waveform\n'
    fid.write(header)
    # write waveforms to csv
    for channel,index in wf_mapping.items():
        # skip channels without good waveforms
        if index is None:
            continue
        # write using an f-string. Found that easiest for 
        row = f"{channel}, ,{FR_df.iloc[index]['template_wf'].tolist()}\n".replace('[','').replace(']','')
        fid.write(row)

# FR at 2ms for different currents at 400 um

In [ ]:
base_dirs = ['Z://BrainPatch//20241002//lateral//',
             'Z://BrainPatch//20240925//',
             'Z:BrainPatch//20240821']

# all 2ms stimulations at 400 um in the base_dirs
directories = [os.path.join(base_dir,directory) for base_dir in base_dirs for directory in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir,directory)) and '2ms' in directory and '600um' in directory]

# probe map
probe_name = "Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat"

# settings for kilosort
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

FR_df = gimme_a_FR_df(directories, probe_name, settings) 





In [40]:
fig_amp_scatter, ax_amp_scatter = plt.subplots(nrows = 4, ncols=2, sharex=True, sharey=True)
fig_amp_line, ax_amp_line = plt.subplots(nrows = 4, ncols = 2, sharex=True, sharey=True)

fig_amp_line.set_size_inches(6, 10)

fid_response = open('Z:\\BrainPatch\\Current_vs_responses_stim_responses.csv','w')
fid_means = open('Z:\\BrainPatch\\Current_vs_responses_prestim_means.csv','w')

for i_current, current in enumerate(FR_df['current'].unique()):
    # first 2ms firing rate -- scatter
    FR_df.loc[FR_df['current'].eq(current)].plot.scatter(ax = ax_amp_scatter[i_current, 1], x = 'poststim_first', y = 'depth', s=2)
    ax_amp_scatter[i_current,1].set_title(f'{current} mA stimulation responses')
    
    # line plot of response mean and std 
    summary = FR_df.loc[FR_df['current'].eq(current)].groupby('depth')['poststim_first'].agg(['mean','std']) #.plot(ax = ax_amp_line[i_current, 1], x = 'mean', y='depth', xerr = 'std')
    ax_amp_line[i_current, 1].plot(summary['mean'], summary.index)
    ax_amp_line[i_current, 1].fill_betweenx(summary.index, np.maximum(summary['mean'] - summary['std'],0), summary['mean'] + summary['std'], alpha=0.2)
    ax_amp_line[i_current,1].set_title(f'{current} mA stimulation responses')

    # stim to csv
    summary['current'] = current
    summary.to_csv(fid_response, header=(i_current==0))


    # pre-stimulation mean
    FR_df.loc[FR_df['current'].eq(current)].plot.scatter(ax = ax_amp_scatter[i_current, 0], x = 'prestim_mean', y = 'depth', s=2)
    ax_amp_scatter[i_current,0].set_title(f'{current} mA pre-stimulation means')
    
    # line plot of mean and std
    summary = FR_df.loc[FR_df['current'].eq(current)].groupby('depth')['prestim_mean'].agg(['mean','std']) #.plot(ax = ax_amp_line[i_current, 1], x = 'mean', y='depth', xerr = 'std')
    ax_amp_line[i_current, 0].plot(summary['mean'], summary.index)
    ax_amp_line[i_current, 0].fill_betweenx(summary.index, np.maximum(summary['mean'] - summary['std'],0), summary['mean'] + summary['std'], alpha=0.2)
    ax_amp_line[i_current,0].set_title(f'{current} mA pre-stimulation means')
    ax_amp_line[i_current,0].set_ylabel('Depth $\mu$m')

    # stim to csv
    summary['current'] = current
    summary.to_csv(fid_means, header=(i_current==0))
    
    # remove the spines from the axes
    for spine in ax_amp_line[i_current,0].spines:
        ax_amp_line[i_current,0].spines[spine].set_visible(False)
        ax_amp_line[i_current,1].spines[spine].set_visible(False)

fid_means.close()
fid_response.close()

fig_amp_line.savefig('Z://BrainPatch//current_vs_response.svg')

# Firing rate histogram
Single condition with the mean subtracted

In [ ]:
# directory = 'Z:\\BrainPatch\\20241002\\lateral\\Crimson__2024-10-02_12-37-44__15mA_2ms_400um'
base_dirs = ['Z://BrainPatch//20241002//lateral//',
             'Z://BrainPatch//20240925//',
             'Z:BrainPatch//20240821']

# all 2ms stimulations at 400 um in the base_dirs
directories = [os.path.join(base_dir,directory) for base_dir in base_dirs for directory in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir,directory)) and '2ms' in directory and '400um' in directory and '15mA' in directory]

# probe map
probe_name = "Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat"

# settings for kilosort
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

FR_df = gimme_a_FR_df(directories, probe_name, settings)


In [82]:
fig_hist, ax_hist = plt.subplots(ncols=2, sharey=True)

# poststim_hist,bins,_= ax_hist[0].hist(FR_df['poststim_first'], bins=50, cumulative=True, histtype='step')
poststim_hist,bins,_ = ax_hist[0].hist(FR_df['poststim_first'], bins=50, histtype = 'step')
ax_hist[0].set_title('Post-stimulation mean firing rates')
ax_hist[0].set_xlabel('Firing rate (Hz, 2ms bin post-stimulation)')
# prestim_hist,_,_  = ax_hist[1].hist(FR_df['prestim_mean'], bins=bins, cumulative=True, histtype='step')
prestim_hist,_,_ = ax_hist[1].hist(FR_df['prestim_mean'], bins=bins, histtype = 'step') 
ax_hist[1].set_title('Pre-stimulation mean firing rates')
ax_hist[1].set_xlabel('Firing rate (Hz), 2ms bins')

for ax_ in ax_hist:
    for spine in ax_.spines:
        ax_.spines[spine].set_visible(False)

    ax_.set_ylabel('Number of units')

hist_df = pd.DataFrame([prestim_hist.T, poststim_hist.T]).T.set_index(bins[:-1]).rename(columns={0:'prestim_hist',1:'poststim_hist'})
with open('Z:\\BrainPatch\\15mA_400um_FR_hist.csv','w') as fid:
    hist_df.to_csv(fid)



# Basic Raster plots

seems to be with more-or-less the same data?

In [52]:
# base_dir = 'Z:\\BrainPatch\\Raw_Data\\20241002'
base_dir = 'Z:\\BrainPatch\\Raw_Data\\20240925'
# base_dir = 'Z:\\BrainPatch\\20240821'

kilosort_directories = [os.path.join(base_dir, directory) for directory in os.listdir(base_dir)
                        if all([string in directory for string in ['2ms','400um','15mA']])]


# kilosort_directories = [os.path.join(base_dir, directory) for directory in 
#                         ['Crimson__2024-10-02_12-26-46__20mA_2ms_400um',
#                         'Crimson__2024-10-02_12-37-44__15mA_2ms_400um',
#                         'Crimson__2024-10-02_12-41-41__10mA_2ms_400um',
#                         'Crimson__2024-10-02_12-49-56__5mA_2ms_400um']]

fig_psth, ax_psth = plt.subplots(nrows = len(kilosort_directories), sharex=True)
# fig_avg, ax_avg = plt.subplots(nrows=len(kilosort_directories), sharex = True)

n_stims = 5

if len(kilosort_directories) == 1:
    ax_psth = np.array(ax_psth, ndmin=1)

for i_directory,directory in enumerate(kilosort_directories):
    kilosort_dir = os.path.join(directory, 'kilosort4_filtered')

    current = re.search('(\d{1,2})mA',directory)[1]
    
    template_times = np.load(os.path.join(kilosort_dir, 'spike_times.npy'))
    template_numbers = np.load(os.path.join(kilosort_dir, 'spike_templates.npy'))
    stim_times = np.load(os.path.join(directory, 'stim_sample_nums.npy'))

    ROI = np.nonzero((template_times > (stim_times[0,0]-60000)) & (template_times < stim_times[n_stims,0])) # get everything up to the Nth stimulation
    template_times = template_times[ROI]
    template_numbers = template_numbers[ROI]
    template_times = (template_times - template_times[0])/30000

    for template in np.unique(template_numbers):
        ax_psth[i_directory].vlines(template_times, template, template + 1)

    n_templates = len(np.unique(template_numbers))
    for i_stim in range(n_stims):
        ax_min,ax_max = ax_psth[i_directory].get_ylim()
        stim_start,stim_end = (stim_times[i_stim,:] - template_times[0])/30000
        stim_patch_array = np.array([[stim_start,ax_min],[stim_end,ax_max],[stim_end,ax_max],[stim_start,ax_min]])
        ax_psth[i_directory].add_patch(Polygon(stim_patch_array, alpha=0.3, color='k'))
    
    for spine in ax_psth[i_directory].spines:
        ax_psth[i_directory].spines[spine].set_visible(False)

    ax_psth[i_directory].set_xlabel('Time (s)')
    ax_psth[i_directory].set_ylabel('Unit')
    # ax_psth[i_directory].set_title(directory)


In [53]:
with open('Z:\\BrainPatch\\SpikeTimes.csv', 'w') as fid:
    csv_writer = csv.writer(fid)

    csv_writer.writerow(['Unit','Time'])
    csv_writer.writerows(list(zip(template_numbers, template_times)))

# Comparison between pre-stim and stimulation

Compute t-tests, and plot on a scatter between the pre-stim and post-stim mean FR.
color the dots by the distance

In [23]:
# 
base_dirs = ['Z://BrainPatch//Raw_Data//20241002',
             'Z://BrainPatch//Raw_Data//20240925',
             'Z:BrainPatch//Raw_Data//20240821']

# all 2ms stimulations at 400 um in the base_dirs
# directories = [os.path.join(base_dir,directory) for base_dir in base_dirs for directory in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir,directory)) and '2ms' in directory and '400um' in directory and '15mA' in directory]
directories = [os.path.join(base_dir,directory) for base_dir in base_dirs for directory in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir,directory)) and '2ms' in directory and '15mA' in directory]

# probe map
probe_name = "Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat"

# settings for kilosort
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

FR_df = gimme_a_FR_df(directories, probe_name, settings)


  0%|          | 0/16 [00:00<?, ?it/s]

15mA 400mm
loading previously converted files Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_12-35-37__15mA_2ms_400um\sig_raw.npy
Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_12-35-37__15mA_2ms_400um\kilosort4_filtered already exists, using existing files


e:\Kevin\Anaconda\kilosort\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


15mA 400mm
loading previously converted files Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_12-37-44__15mA_2ms_400um\sig_raw.npy
Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_12-37-44__15mA_2ms_400um\kilosort4_filtered already exists, using existing files
15mA 400mm
loading previously converted files Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_12-52-58__15mA_2ms_400um\sig_raw.npy
Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_12-52-58__15mA_2ms_400um\kilosort4_filtered already exists, using existing files
15mA 600mm
loading previously converted files Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_13-20-08__15mA_2ms_600um\sig_raw.npy
Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_13-20-08__15mA_2ms_600um\kilosort4_filtered already exists, using existing files
15mA 900mm
loading previously converted files Z://BrainPatch//Raw_Data//20241002\Crimson__2024-10-02_13-39-44__15mA_2ms_900um\sig_raw.npy
Z://BrainPatch//Raw_Data//20241002\Crims

In [5]:
# list to keep track of the unit # etc
def significance_value(sig_tuple:tuple):
    return sig_tuple[1]


sig_tups = FR_df['significant'].apply(significance_value)

In [ ]:
np.sum(sig_tups<0.01)/len(sig_tups)

Plot the average pre-stim vs the average 2ms post-stim on a log-log plot

In [7]:
fig_scatter_mean, ax_scatter_mean = plt.subplots()

FR_df['prestim_mean'] = FR_df['prestim_mean'] + 0.01
FR_df['poststim_first'] = FR_df['poststim_first'] + 0.01

colors = ['red' if significance[1] < 0.01 else 'gray' for significance in FR_df['significant']]


ax_scatter_mean.scatter(FR_df['prestim_mean'], FR_df['poststim_first'], 2, colors)

ax_scatter_mean.set_xlabel('Pre-stimulus mean firing rate (Hz)')
ax_scatter_mean.set_ylabel('Mean stimulus response (Hz, 2ms post-stim bin)')

for spine in ax_scatter_mean.spines:
    ax_scatter_mean.spines[spine].set_visible(False)

# ax_scatter_mean.set_yscale('log')
# ax_scatter_mean.set_xscale('log')

ax_scatter_mean.set_xlim(ax_scatter_mean.get_ylim())


ax_scatter_mean.plot(ax_scatter_mean.get_xlim(), ax_scatter_mean.get_ylim(), 'k')
ax_scatter_mean.set_aspect(aspect='equal', adjustable='box')

In [90]:
distances = FR_df.loc[~FR_df['distance'].astype(int).isin([711, 794, 1007, 1068, 1305, 1353, 1604, 1643])]['distance'].unique()
distances.sort()
fig_delta_mean, ax_delta_mean = plt.subplots(ncols = len(distances), sharey=True)
# fig_delta_mean, ax_delta_mean = plt.subplots(ncols=2, sharey=True)

for i_distance,distance in enumerate(distances):
    # if int(distance) in [711, 794, 1007, 1068, 1305, 1353, 1604, 1643]:
    #     continue
    temp_df = FR_df.loc[FR_df['distance'] == distance].copy()
    temp_df = temp_df.loc[(temp_df['prestim_mean'] > .01) & (temp_df['poststim_first'] > .01)]
    temp_df[['prestim_mean','poststim_first']].dropna()

    pre_post = np.array(temp_df[['prestim_mean', 'poststim_first']])
    horiz_ = np.zeros(pre_post.shape)
    horiz_[:,1] = 1

    colors = ['red' if significance[1] < 0.01 else 'gray' for significance in temp_df['significant']]

    for i_color,color in enumerate(colors):
        ax_delta_mean[i_distance].plot(horiz_[i_color,:], pre_post[i_color,:], color=color, alpha = 0.2)

    ax_delta_mean[i_distance].set_yscale('log')


    for spine in ax_delta_mean[i_distance].spines:
        ax_delta_mean[i_distance].spines[spine].set_visible(False)

    ax_delta_mean[i_distance].set_title(f'{int(distance)} $\mu$m')

    ax_delta_mean[i_distance].set_xticks([0,1])
    ax_delta_mean[i_distance].set_xticklabels(['Pre-stimulation mean','Post-stimulation response (2ms bin)'])
    ax_delta_mean[i_distance].tick_params(axis='x', rotation=45)

    if i_distance > 0:
        ax_delta_mean[i_distance].set_yticklabels([])

ax_delta_mean[0].set_ylabel('Firing Rate (Hz)')

Text(0, 0.5, 'Firing Rate (Hz)')

In [95]:
distances = FR_df.loc[~FR_df['distance'].astype(int).isin([711, 794, 1007, 1068, 1305, 1353, 1604, 1643])]['distance'].unique()
distances.sort()
fig_delta_mean, ax_delta_mean = plt.subplots()

for i_distance,distance in enumerate(distances):
    # if int(distance) in [711, 794, 1007, 1068, 1305, 1353, 1604, 1643]:
    #     continue
    temp_df = FR_df.loc[FR_df['distance'] == distance].copy()
    temp_df = temp_df.loc[(temp_df['prestim_mean'] > .01) & (temp_df['poststim_first'] > .01)]
    temp_df[['prestim_mean','poststim_first']].dropna()

    pre_post = np.array(temp_df[['prestim_mean', 'poststim_first']])
    horiz_ = np.zeros(pre_post.shape)
    horiz_[:,1] = 1
    horiz_ = horiz_ + 2*i_distance

    colors = ['red' if significance[1] < 0.01 else 'gray' for significance in temp_df['significant']]

    for i_color,color in enumerate(colors):
        ax_delta_mean.plot(horiz_[i_color,:], pre_post[i_color,:], color=color, alpha = 0.2)

    ax_delta_mean.set_yscale('log')


    for spine in ax_delta_mean.spines:
        ax_delta_mean.spines[spine].set_visible(False)

    # ax_delta_mean.set_title(f'{int(distance)} $\mu$m')

ax_delta_mean.set_xticks(np.arange(start=0.5, step=2, stop = 2*len(distances)))
ax_delta_mean.set_xticklabels([f"{int(distance)} $\mu$" for distance in distances])
    # ax_delta_mean.tick_params(axis='x', rotation=45)

ax_delta_mean.set_ylabel('Firing Rate (Hz)')

Text(0, 0.5, 'Firing Rate (Hz)')

In [82]:
~FR_df['distance'].isin([711, 794, 1007, 1068, 1305, 1353, 1604, 1643])

0       True
1       True
2       True
3       True
4       True
        ... 
1690    True
1691    True
1692    True
1693    True
1694    True
Name: distance, Length: 1695, dtype: bool

In [60]:
FR_df.head()

,template,channel,prestim_mean,prestim_std,poststim_max,poststim_first,type,distance,depth,current,shank_no,template_wf,mean_firing_rate,recording,significant
0,0,41,2.375297,34.380320,287.500000,0.000000,excited,548.292805,-0.0,15,1.0,"[-0.752717, -0.96794075, -1.0633813, -1.115241...","[0.0, 287.5, 245.83333333333334, 170.833333333...",Z://BrainPatch//Raw_Data//20241002\Crimson__20...,"(2.4543566058029724, 0.9928760117146457)"
1,1,41,1.583531,28.093737,308.333333,16.666667,excited,548.292805,-0.0,15,1.0,"[-0.5542434, -0.7080297, -0.8477048, -0.905614...","[16.666666666666668, 220.83333333333334, 308.3...",Z://BrainPatch//Raw_Data//20241002\Crimson__20...,"(-1.8248215984772367, 0.035245290327122925)"
2,2,34,0.395883,14.063594,379.166667,4.166667,low_firing,548.292805,-50.0,15,1.0,"[-0.660214, -0.84989595, -1.0503551, -1.168656...","[4.166666666666667, 154.16666666666666, 379.16...",Z://BrainPatch//Raw_Data//20241002\Crimson__20...,"(-0.9009307928289398, 0.18470624908891986)"
3,3,34,1.187648,24.339551,454.166667,20.833333,excited,548.292805,-50.0,15,1.0,"[0.3601714, 0.40841317, 0.47471786, 0.501565, ...","[20.833333333333332, 287.5, 454.1666666666667,...",Z://BrainPatch//Raw_Data//20241002\Crimson__20...,"(-2.1389775867022327, 0.0172290211935325)"
4,4,41,2.771180,37.120216,475.000000,16.666667,excited,548.292805,-0.0,15,1.0,"[0.7561821, 0.984202, 1.1551046, 1.2790855, 1....","[16.666666666666668, 320.8333333333333, 466.66...",Z://BrainPatch//Raw_Data//20241002\Crimson__20...,"(-1.6754251654948362, 0.048197010702231494)"


In [51]:
(temp_df['prestim_mean'] == 0) | (temp_df['poststim_first'] == 0)

5       False
6       False
7        True
8       False
9       False
        ...  
1562     True
1565     True
1566    False
1569     True
1570    False
Length: 275, dtype: bool

In [34]:
pre_post

array([[2.11091921, 0.        ],
       [3.64613318, 0.        ],
       [2.87852619, 4.16666667],
       [0.09595087, 2.08333333],
       [0.        , 0.        ],
       [0.09595087, 0.        ],
       [8.44367684, 4.16666667],
       [0.38380349, 0.        ],
       [3.83803493, 0.        ],
       [2.97447707, 0.        ],
       [0.47975437, 0.        ],
       [0.09595087, 2.08333333],
       [2.20687008, 0.        ],
       [3.26232969, 0.        ],
       [0.86355786, 0.        ],
       [3.55018231, 4.16666667],
       [0.        , 0.        ],
       [0.38380349, 0.        ]])

In [20]:
high_firing = FR_df.loc[FR_df['poststim_first'] > 250]

ncols,nrows = np.floor(np.sqrt(high_firing.shape[0])).astype(int), np.ceil(np.sqrt(high_firing.shape[0])).astype(int)
fig,ax = plt.subplots(nrows=nrows,ncols=ncols)

# ax.plot(FR_df.loc[FR_df['poststim_first'] > 500]['template_wf'])
for i_waveform,waveform in enumerate(FR_df.loc[FR_df['poststim_first'] > 250]['template_wf']):
    ax.ravel()[i_waveform].plot(waveform)
    ax.ravel()[i_waveform].set_title(f"{high_firing.iloc[i_waveform]['channel']}: {high_firing.iloc[i_waveform]['poststim_first'].astype(int)}")

# spike times vs firing rates -- checking to make sure my binning is really working the way I think it is.

In [ ]:
probe_name = "Z:\\BrainPatch\\20241002\\64-4shank-poly-brainpatch-chanMap.mat"

# settings for kilosort
settings = {'probe_name':probe_name,
            'n_chan_bin':64,
            'nearest_chans':1}

directory = []

FR_df_single = gimme_a_FR_df(directory, probe_name, settings)